In [2]:
pip install pyathena


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
from datetime import datetime

In [4]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [5]:
#Connection to daily_check (based on receiver)
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_1607/2235117015.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [6]:
#Connection to daily_forex 
forex_table = 'last_daily_forex'
database_name= 'viamericas'
df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)

/tmp/ipykernel_1607/3034134324.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)


In [7]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [8]:
df = df[df['date'] != '2020-12-31'] # Excluyo el 31-12-2020
df = df[df['date'] <= '2022-11-25'] # Cortamos el dataset hasta el 25/11/2022

In [9]:
df.date.max()

Timestamp('2022-11-25 00:00:00')

In [10]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [11]:
df['country'].unique()

array(['ECUADOR', 'HONDURAS', 'EL SALVADOR', 'UNITED STATES',
       'PHILIPPINES', 'MEXICO', 'COSTA RICA', 'HAITI', 'NIGERIA',
       'GUATEMALA', 'PERU', 'UNITED STATES (ATM & DEPOSITS)',
       'DOMINICAN REPUBLIC', 'BOLIVIA', 'CHILE', 'BRAZIL', 'COLOMBIA',
       'THAILAND', 'GHANA', 'NICARAGUA', 'SENEGAL', 'BENIN',
       'SIERRA LEONE', 'INDONESIA', 'BANGLADESH', 'ARGENTINA', 'GUINEA',
       'SPAIN', 'INDIA', 'RWANDA', 'FRANCE', 'URUGUAY',
       "COTE D'IVOIRE (IVORY COAST)", 'CAMEROON', 'TOGO', 'NETHERLANDS',
       'PORTUGAL', 'NEPAL', 'KENYA', 'TANZANIA', 'BURKINA FASO',
       'GAMBIA, THE', 'LIBERIA', 'MALI', 'MALAYSIA', 'ZAMBIA', 'ROMANIA',
       'GERMANY', 'BELGIUM', 'UNITED KINGDOM', 'ITALY', 'VIETNAM',
       'AUSTRIA', 'SWITZERLAND', 'IRELAND', 'GREECE', 'UGANDA',
       'ETHIOPIA', 'SRI LANKA', 'GABON'], dtype=object)

In [12]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA - CHEQUEAR!

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_1607/3736126815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [13]:
# FOREX - Selecting columns & renaming
df_rates=df_rates.rename(columns={'day': 'date', 'max_feed_price': 'feed_price'})
df_rates=df_rates.loc[:,['date', 'feed_price', 'symbol']]

In [14]:
df_rates

,date,feed_price,symbol
0,2022-04-03,75.9958,USDINR
1,2022-04-03,4.6608,USDBRL
2,2022-04-03,19.8723,USDMXN
3,2022-04-03,51.5945,USDPHP
4,2022-04-03,7.6989,USDGTQ
...,...,...,...
5120,2022-03-02,5.1883,USDBRL
5121,2022-03-02,51.6402,USDPHP
5122,2022-03-02,7.7180,USDGTQ
5123,2022-03-02,20.7219,USDMXN


In [15]:
def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' > chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [16]:
start_date = '2021-01-01'
end_date = '2022-11-25'

rates = get_closing_prices(df_rates, start_date, end_date)

In [17]:
###WE OBTAIN THE LAGS FOR THE EXCHANGE RATE 

def generate_lag_and_variation(df, num_lags):
    # Create columns for each day's lag up to the defined maximum
    for i in range(1, num_lags + 1):
        col_name = f'rate_lag_{i}'
        # Shift the 'feed_price' column grouped by 'symbol'
        df[col_name] = df.groupby('symbol')['feed_price'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, num_lags):
        col_name = f'var_rate_lag_{i}'
        # Calculate the difference between consecutive delay columns
        df[col_name] = df[f'rate_lag_{i}'] - df[f'rate_lag_{i + 1}']
    
    return df

In [18]:
rates_number = 14
rates = generate_lag_and_variation(rates, rates_number)
rates['date'] = rates['date'].dt.date #Date format

In [19]:
rates

,symbol,date,feed_price,rate_lag_1,rate_lag_2,rate_lag_3,rate_lag_4,rate_lag_5,rate_lag_6,rate_lag_7,...,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,var_rate_lag_7,var_rate_lag_8,var_rate_lag_9,var_rate_lag_10,var_rate_lag_11,var_rate_lag_12,var_rate_lag_13
0,USDBRL,2021-01-01,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USDBRL,2021-01-02,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDBRL,2021-01-03,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDBRL,2021-01-04,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDBRL,2021-01-05,5.3244,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,USDPHP,2022-11-21,57.4550,57.2855,57.2262,57.4140,57.5554,57.5601,57.4392,57.3329,...,-0.0047,0.1209,0.1063,0.0314,0.0293,-0.0715,-0.7152,-0.1787,-0.0976,-0.1260
3466,USDPHP,2022-11-22,57.4507,57.4550,57.2855,57.2262,57.4140,57.5554,57.5601,57.4392,...,-0.1414,-0.0047,0.1209,0.1063,0.0314,0.0293,-0.0715,-0.7152,-0.1787,-0.0976
3467,USDPHP,2022-11-23,57.2812,57.4507,57.4550,57.2855,57.2262,57.4140,57.5554,57.5601,...,-0.1878,-0.1414,-0.0047,0.1209,0.1063,0.0314,0.0293,-0.0715,-0.7152,-0.1787
3468,USDPHP,2022-11-24,56.7979,57.2812,57.4507,57.4550,57.2855,57.2262,57.4140,57.5554,...,0.0593,-0.1878,-0.1414,-0.0047,0.1209,0.1063,0.0314,0.0293,-0.0715,-0.7152


In [20]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [21]:
rates['country'] = rates['symbol'].map(rates_dict)

In [22]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3470 entries, 0 to 3469
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           3470 non-null   object 
 1   date             3470 non-null   object 
 2   feed_price       3470 non-null   float64
 3   rate_lag_1       3465 non-null   float64
 4   rate_lag_2       3460 non-null   float64
 5   rate_lag_3       3455 non-null   float64
 6   rate_lag_4       3450 non-null   float64
 7   rate_lag_5       3445 non-null   float64
 8   rate_lag_6       3440 non-null   float64
 9   rate_lag_7       3435 non-null   float64
 10  rate_lag_8       3430 non-null   float64
 11  rate_lag_9       3425 non-null   float64
 12  rate_lag_10      3420 non-null   float64
 13  rate_lag_11      3415 non-null   float64
 14  rate_lag_12      3410 non-null   float64
 15  rate_lag_13      3405 non-null   float64
 16  rate_lag_14      3400 non-null   float64
 17  var_rate_lag_1

In [23]:
df1 = pd.merge(df_filtered, rates, on=['date', 'country'], how='left')
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [24]:
### EFFECT OF CANCELED TRANSACTIONS ###
database_name= 'analytics'
table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS

df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[df2['date'] != '2020-12-31'] # Elimino el registro del 31.12.2020

/tmp/ipykernel_1607/1901381988.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)


In [25]:
from sklearn.preprocessing import LabelEncoder

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df2['payer_country'] = df2['payer'] + '_' + df2['country']
# Inicializing LabelEncoder
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df2['payer_country_encoder'] = label_encoder.fit_transform(df2['payer_country'])

In [26]:
# By applying the same aging filter, we can work on the same payer_country universe 
df2 = df2[df2['payer_country'].isin(df_aging['payer_country'])]

In [27]:
def fill_missing_dates(df, start_date, end_date):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=start_date, end=end_date)
    df_fill = pd.DataFrame({'date': date_range, 'amount': 0, 'tx_cancelled': 0})
    df_fill['date'] = pd.to_datetime(df_fill['date']).dt.date
    
    # Sort the original DataFrame by 'country', 'payer', and 'date'
    df = df.sort_values(by=['country', 'payer', 'date'])
    
    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'payer_country'
    for payer_country in df['payer_country'].unique():
#        print(payer_country)
        df_aux = df[df['payer_country'] == payer_country]
        # Combining df_aux (payer_country) with df_fill, keeping values from df_aux and filling dates, amount and tx_cancelled 
        merged_df = df_aux.set_index('date').combine_first(df_fill.set_index('date')).reset_index()

        # Fill missing values in 'payer', 'country', 'payer_country', 'payer_country_encoder' columns
        columns_to_fill = ['payer', 'country', 'payer_country', 'payer_country_encoder']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    
    return result_df

In [28]:
#start_date and end_date variables defined previously
start_date = '2021-01-01'
end_date = '2022-11-25'

# Call the function with the specified start_date and end_date
df_full = fill_missing_dates(df2, start_date, end_date)

In [29]:
def generate_tx_lags_and_variation(df, tx_count):
    """
    Generate lag columns for cancelled transactions and their variations.

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_cancelled_lag_{i}'
        # Shift the 'tx_cancelled' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx_cancelled'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, tx_count):
        col_name = f'var_tx_cancelled_lag_{i}'
        # Calculate the difference between consecutive lag columns
        df[col_name] = df[f'tx_cancelled_lag_{i}'] - df[f'tx_cancelled_lag_{i + 1}']

    return df


In [30]:
# Call the function and assign the result back to df2
tx_cancelled_lags = 7
df2 = generate_tx_lags_and_variation(df_full, tx_cancelled_lags)

In [31]:
df2['date'] = df2['date'].dt.date

In [32]:
df2

,date,amount,country,payer,payer_country,payer_country_encoder,tx_cancelled,tx_cancelled_lag_1,tx_cancelled_lag_2,tx_cancelled_lag_3,tx_cancelled_lag_4,tx_cancelled_lag_5,tx_cancelled_lag_6,tx_cancelled_lag_7,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6
0,2021-01-01,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,25.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,2021-01-05,490.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103157,2022-12-29,0.0,URUGUAY,BBVA URUGUAY (BTS),BBVA URUGUAY (BTS)_URUGUAY,48.0,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
103158,2023-01-02,0.0,URUGUAY,BBVA URUGUAY (BTS),BBVA URUGUAY (BTS)_URUGUAY,48.0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
103159,2023-01-04,0.0,URUGUAY,BBVA URUGUAY (BTS),BBVA URUGUAY (BTS)_URUGUAY,48.0,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
103160,2023-01-15,0.0,URUGUAY,BBVA URUGUAY (BTS),BBVA URUGUAY (BTS)_URUGUAY,48.0,1,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'], how='outer')
df_final['date'] = pd.to_datetime(df_final['date'])

In [34]:
df_final.columns

Index(['payer', 'country', 'date', 'tx', 'amount', 'coupon_count',
       'payer_country_x', 'symbol', 'feed_price', 'rate_lag_1', 'rate_lag_2',
       'rate_lag_3', 'rate_lag_4', 'rate_lag_5', 'rate_lag_6', 'rate_lag_7',
       'rate_lag_8', 'rate_lag_9', 'rate_lag_10', 'rate_lag_11', 'rate_lag_12',
       'rate_lag_13', 'rate_lag_14', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6',
       'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
       'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
       'payer_country_y', 'payer_country_encoder', 'tx_cancelled',
       'tx_cancelled_lag_1', 'tx_cancelled_lag_2', 'tx_cancelled_lag_3',
       'tx_cancelled_lag_4', 'tx_cancelled_lag_5', 'tx_cancelled_lag_6',
       'tx_cancelled_lag_7', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_c

In [35]:
###WE SHOULD KEEP THE VARIABLES THAT HAVE IMPUTED MISSINGS IN AMOUNT AND TX_CANCELLED (DAYS WITHOUT OPERATIONS) BUT KEEP THE MISSINGS IN RATES
df_final=df_final.loc[:,['date', 'payer', 'country', 'amount', 
                         'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3',
                         'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7',
                         'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
                         'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
                         'payer_country_encoder','payer_country_y',
                         'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
                         'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6']]
df_final = df_final.rename(columns={'payer_country_y': 'payer_country'})

In [36]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103162 entries, 0 to 103161
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    103162 non-null  datetime64[ns]
 1   payer                   103162 non-null  object        
 2   country                 103162 non-null  object        
 3   amount                  103162 non-null  float64       
 4   var_rate_lag_1          22126 non-null   float64       
 5   var_rate_lag_2          22097 non-null   float64       
 6   var_rate_lag_3          22068 non-null   float64       
 7   var_rate_lag_4          22039 non-null   float64       
 8   var_rate_lag_5          22011 non-null   float64       
 9   var_rate_lag_6          21982 non-null   float64       
 10  var_rate_lag_7          21954 non-null   float64       
 11  var_rate_lag_8          21927 non-null   float64       
 12  var_rate_lag_9          21898 

In [37]:
### DUMMIES ####

# Weekend indicator (dummy)
df_final['weekend'] = (df_final['date'].dt.dayofweek >= 4).astype(int)

In [38]:
#WE ADD ONE MORE DAY AFTER CHRISTMAS, NEW YEAR'S EVE AND US INDEPENDENCE (BETTER CAPTURES THE DISTURBANCE).
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26',
                     '07-05'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [39]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [40]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [41]:
df_final['amount'].sum()

24526093622.4718

In [42]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103162 entries, 0 to 103161
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    103162 non-null  datetime64[ns]
 1   payer                   103162 non-null  object        
 2   country                 103162 non-null  object        
 3   amount                  103162 non-null  float64       
 4   var_rate_lag_1          22126 non-null   float64       
 5   var_rate_lag_2          22097 non-null   float64       
 6   var_rate_lag_3          22068 non-null   float64       
 7   var_rate_lag_4          22039 non-null   float64       
 8   var_rate_lag_5          22011 non-null   float64       
 9   var_rate_lag_6          21982 non-null   float64       
 10  var_rate_lag_7          21954 non-null   float64       
 11  var_rate_lag_8          21927 non-null   float64       
 12  var_rate_lag_9          21898 

In [43]:
# Impute missings in exogenous variables
df_final.fillna(0, inplace=True)

In [44]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103162 entries, 0 to 103161
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    103162 non-null  datetime64[ns]
 1   payer                   103162 non-null  object        
 2   country                 103162 non-null  object        
 3   amount                  103162 non-null  float64       
 4   var_rate_lag_1          103162 non-null  float64       
 5   var_rate_lag_2          103162 non-null  float64       
 6   var_rate_lag_3          103162 non-null  float64       
 7   var_rate_lag_4          103162 non-null  float64       
 8   var_rate_lag_5          103162 non-null  float64       
 9   var_rate_lag_6          103162 non-null  float64       
 10  var_rate_lag_7          103162 non-null  float64       
 11  var_rate_lag_8          103162 non-null  float64       
 12  var_rate_lag_9          103162

In [45]:
##mantenemos mismos id_payer que antes##
del df_final['payer_country_encoder']
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df_final['payer_country_encoder'] = label_encoder.fit_transform(df_final['payer_country'])

In [46]:
# Setting DS timeframe

df_25_11 = df_final.loc[df_final.date<'2022-11-26']


In [47]:
df_25_11['amount'].sum()

14178037673.089

In [48]:
df_25_11.columns

Index(['date', 'payer', 'country', 'amount', 'var_rate_lag_1',
       'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9',
       'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'payer_country', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_cancelled_lag_6', 'weekend', 'special_dates',
       'payer_country_encoder'],
      dtype='object')

In [49]:
df_25_11.date.max(),df_25_11.date.min()

(Timestamp('2022-11-25 00:00:00'), Timestamp('2021-01-01 00:00:00'))

In [50]:
##Top 10 payers##
#total_amount_per_payer = df_final.groupby('payer_country_encoder')['amount'].sum()
#top_10_payers = total_amount_per_payer.sort_values(ascending=False).head(10)
#print(top_10_payers)

In [51]:
#top_10_payers=[57,29,36,24,38,60,32,20,19,101]
#top_10 = df_final[df_final['payer_country_encoder'].isin(top_10_payers)]

In [52]:
#top_10.country.unique()

In [53]:
#top_10.loc[top_10.country=='HONDURAS']

In [54]:
df_25_11

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
0,2022-06-27,ECUAGIROS,ECUADOR,106780.51,0.0,0.0,0.0,0.0,0.0,0.0,...,ECUAGIROS_ECUADOR,-11.0,3.0,-17.0,14.0,-3.0,-5.0,0,0,49
1,2022-07-28,COOPERATIVA FRONTERIZA (UT),HONDURAS,12747.00,0.0,0.0,0.0,0.0,0.0,0.0,...,COOPERATIVA FRONTERIZA (UT)_HONDURAS,3.0,0.0,-5.0,5.0,-1.0,1.0,0,0,42
2,2022-02-01,ABANK (TN),EL SALVADOR,16275.00,0.0,0.0,0.0,0.0,0.0,0.0,...,ABANK (TN)_EL SALVADOR,0.0,1.0,-5.0,2.0,5.0,-2.0,0,0,0
3,2022-08-14,COOPERATIVA CACIL (UT),HONDURAS,10516.00,0.0,0.0,0.0,0.0,0.0,0.0,...,COOPERATIVA CACIL (UT)_HONDURAS,1.0,0.0,-1.0,0.0,-4.0,2.0,1,0,41
4,2022-07-04,VIAMERICAS USA DEPOSITS,UNITED STATES,40435.04,0.0,0.0,0.0,0.0,0.0,0.0,...,VIAMERICAS USA DEPOSITS_UNITED STATES,-5.0,-8.0,3.0,11.0,2.0,-14.0,0,1,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103141,2022-11-18,BBVA URUGUAY (BTS),URUGUAY,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,BBVA URUGUAY (BTS)_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,1,0,36
103142,2022-11-20,BBVA URUGUAY (BTS),URUGUAY,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,BBVA URUGUAY (BTS)_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,1,0,36
103143,2022-11-22,BBVA URUGUAY (BTS),URUGUAY,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,BBVA URUGUAY (BTS)_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,0,0,36
103144,2022-11-23,BBVA URUGUAY (BTS),URUGUAY,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,BBVA URUGUAY (BTS)_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,0,0,36


In [55]:
control=df_final.loc[df_final.payer_country_encoder==57]

In [56]:
##Elektra volvió a ser el 57##
control

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
2676,2022-04-08,FEDECACES (EL SALVADOR),EL SALVADOR,165949.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,3.0,3.0,1.0,2.0,-2.0,-2.0,1,0,57
2752,2021-09-13,FEDECACES (EL SALVADOR),EL SALVADOR,143404.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,4.0,-4.0,3.0,-4.0,4.0,-2.0,0,0,57
4212,2021-01-28,FEDECACES (EL SALVADOR),EL SALVADOR,109481.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,2.0,-2.0,-4.0,-3.0,10.0,1.0,0,0,57
5345,2021-01-01,FEDECACES (EL SALVADOR),EL SALVADOR,10218.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,0.0,0.0,0.0,0.0,0.0,0.0,1,0,57
5810,2021-02-08,FEDECACES (EL SALVADOR),EL SALVADOR,122627.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,6.0,-5.0,6.0,-4.0,-2.0,2.0,0,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67778,2023-10-17,FEDECACES (EL SALVADOR),EL SALVADOR,180377.32,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,3.0,-6.0,2.0,4.0,-6.0,-4.0,0,0,57
67779,2023-10-18,FEDECACES (EL SALVADOR),EL SALVADOR,125490.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,2.0,3.0,-6.0,2.0,4.0,-6.0,0,0,57
67780,2023-10-19,FEDECACES (EL SALVADOR),EL SALVADOR,146593.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,-4.0,2.0,3.0,-6.0,2.0,4.0,0,0,57
67781,2023-10-20,FEDECACES (EL SALVADOR),EL SALVADOR,231622.00,0.0,0.0,0.0,0.0,0.0,0.0,...,FEDECACES (EL SALVADOR)_EL SALVADOR,6.0,-4.0,2.0,3.0,-6.0,2.0,1,0,57


In [57]:
# TARGET CSV
target_ts_amount=df_25_11.loc[:, ['payer_country_encoder', 'date', 'amount']]
target_ts_amount.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp', 'amount': 'target_value'}, inplace=True)
target_ts_amount.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72870 entries, 0 to 103145
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_id       72870 non-null  int64         
 1   timestamp     72870 non-null  datetime64[ns]
 2   target_value  72870 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.2 MB


In [58]:
# RELATED TS
###GENERATE THE DATASET OF RELATED TIME SERIES (UP TO 23 VARIABLES+ID_ITEM+TIMESTAMP-FIRST THESE TWO VARIABLES)
related_ts_columns = ['payer_country_encoder', 'date', 
                      'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend', 'special_dates']

related_ts_ds = df_25_11[related_ts_columns]
related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)

/tmp/ipykernel_1607/1790408907.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)


In [59]:
related_ts_ds.columns

Index(['item_id', 'timestamp', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6',
       'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
       'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
       'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4',
       'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend',
       'special_dates'],
      dtype='object')

In [60]:
related_ts_ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72870 entries, 0 to 103145
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   item_id                 72870 non-null  int64         
 1   timestamp               72870 non-null  datetime64[ns]
 2   var_rate_lag_1          72870 non-null  float64       
 3   var_rate_lag_2          72870 non-null  float64       
 4   var_rate_lag_3          72870 non-null  float64       
 5   var_rate_lag_4          72870 non-null  float64       
 6   var_rate_lag_5          72870 non-null  float64       
 7   var_rate_lag_6          72870 non-null  float64       
 8   var_rate_lag_7          72870 non-null  float64       
 9   var_rate_lag_8          72870 non-null  float64       
 10  var_rate_lag_9          72870 non-null  float64       
 11  var_rate_lag_10         72870 non-null  float64       
 12  var_rate_lag_11         72870 non-null  float64   

In [61]:
 #[item_id, timestamp, var_rate_lag_1, var_rate_lag_2, var_rate_lag_3, var_rate_lag_4, var_rate_lag_5, var_rate_lag_6, var_rate_lag_7, var_rate_lag_8, var_rate_lag_9, var_rate_lag_10, var_rate_lag_11, var_rate_lag_12, var_rate_lag_13, var_tx_cancelled_lag_1, var_tx_cancelled_lag_2, var_tx_cancelled_lag_3, var_tx_cancelled_lag_4, var_tx_cancelled_lag_5, var_tx_cancelled_lag_6, weekend, special_dates]

In [62]:
###GENERATE A DATASET WITH METADATA(COUNTRY AND PAYER)
metadata_ts=df_25_11.loc[:, ['payer_country_encoder', 'country', 'payer']]
metadata_ts.rename(columns={'payer_country_encoder': 'item_id'}, inplace=True)
metadata_ts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72870 entries, 0 to 103145
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  72870 non-null  int64 
 1   country  72870 non-null  object
 2   payer    72870 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [63]:
pip install awswrangler


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [64]:
#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#
##ROUTINES FOR EXPORTING CSV TO VIAMERICAS S3##
import awswrangler as wr
file_name_target = 'target_abt_14lags_25_11_with_zeros.csv'

wr.s3.to_csv(
    df=target_ts_amount,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/{file_name_target}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/target_abt_14lags_25_11_with_zeros.csv'],
 'partitions_values': {}}

In [65]:
import awswrangler as wr
file_name_ts = 'related_ts_14lags_25_11_with_zeros.csv'

wr.s3.to_csv(
    df=related_ts_ds,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_ts}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/related_ts_14lags_25_11_with_zeros.csv'],
 'partitions_values': {}}

In [66]:
file_name_metadata = 'metadata_14lags_25_11_with_zeros.csv'

wr.s3.to_csv(
    df=metadata_ts,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_metadata}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/metadata_14lags_25_11_with_zeros.csv'],
 'partitions_values': {}}